# Introduction to Datasets
[link_to_source](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-datasets.html)

In [1]:
sc.version

2.3.1

## Create a Dataset
To convert a sequence to a Dataset, call .toDS() on the sequence.

In [2]:
val dataset = Seq(1, 2, 3).toDS()

dataset = [value: int]


[value: int]

In [3]:
dataset.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
+-----+



In [4]:
case class Person(name: String, age: Int)
val spark2 = spark
import spark2.implicits._
val personDS = Seq(Person("Max", 33), Person("Adam", 32), Person("Muller", 62)).toDS()

defined class Person
spark2 = org.apache.spark.sql.SparkSession@36ca9a37
personDS = [name: string, age: int]


[name: string, age: int]

In [5]:
personDS.show()

+------+---+
|  name|age|
+------+---+
|   Max| 33|
|  Adam| 32|
|Muller| 62|
+------+---+



## Create a Dataset from an RDD


In [6]:
val rdd = sc.parallelize(Seq((1, "Spark"), (2, "Databricks")))
val integerDS = rdd.toDS()

rdd = ParallelCollectionRDD[0] at parallelize at <console>:34
integerDS = [_1: int, _2: string]


[_1: int, _2: string]

In [7]:
integerDS.show()

+---+----------+
| _1|        _2|
+---+----------+
|  1|     Spark|
|  2|Databricks|
+---+----------+



## Create a Dataset from a DataFrame

In [8]:
case class Company(name: String, foundingYear: Int, numEmployees: Int)

val spark2 = spark
import spark2.implicits._

val inputSeq = Seq(Company("ABC", 1998, 310), Company("XYZ", 1983, 904), Company("NOP", 2005, 83))
val df = sc.parallelize(inputSeq).toDF()

val companyDS = df.as[Company]

defined class Company
spark2 = org.apache.spark.sql.SparkSession@36ca9a37
inputSeq = List(Company(ABC,1998,310), Company(XYZ,1983,904), Company(NOP,2005,83))
df = [name: string, foundingYear: int ... 1 more field]
companyDS = [name: string, foundingYear: int ... 1 more field]


[name: string, foundingYear: int ... 1 more field]

In [9]:
companyDS.show()

+----+------------+------------+
|name|foundingYear|numEmployees|
+----+------------+------------+
| ABC|        1998|         310|
| XYZ|        1983|         904|
| NOP|        2005|          83|
+----+------------+------------+



In [10]:
val rdd = sc.parallelize(Seq((1, "Spark"), (2, "Databricks"), (3, "Notebook")))
val df = rdd.toDF("Id", "Name")

val dataset = df.as[(Int, String)]
dataset.show()

+---+----------+
| Id|      Name|
+---+----------+
|  1|     Spark|
|  2|Databricks|
|  3|  Notebook|
+---+----------+



rdd = ParallelCollectionRDD[10] at parallelize at <console>:47
df = [Id: int, Name: string]
dataset = [Id: int, Name: string]


[Id: int, Name: string]

## Work with Datasets


### Word Count Example

In [11]:
val wordsDataset = sc.parallelize(Seq("Spark I am your father", "May the spark be with you", "Spark I am your father")).toDS()
val groupedDataset = wordsDataset.flatMap(_.toLowerCase.split(" "))
                                 .filter(_ != "")
                                 .groupBy("value")
val countsDataset = groupedDataset.count()

wordsDataset = [value: string]
groupedDataset = RelationalGroupedDataset: [grouping expressions: [value: string], value: [value: string], type: GroupBy]
countsDataset = [value: string, count: bigint]


[value: string, count: bigint]

In [12]:
countsDataset.show()

+------+-----+
| value|count|
+------+-----+
|father|    2|
|   you|    1|
|  with|    1|
|    be|    1|
|  your|    2|
|   may|    1|
| spark|    3|
|   the|    1|
|     i|    2|
|    am|    2|
+------+-----+



### Join Datasets

In [13]:
case class Employee(name: String, age: Int, departmentId: Int, salary: Double)
case class Department(id: Int, name: String)

case class Record(name: String, age: Int, salary: Double, departmentId: Int, departmentName: String)
case class ResultSet(departmentId: Int, departmentName: String, avgSalary: Double)

val spark2 = spark
import spark2.implicits._

val employeeDataSet1 = sc.parallelize(Seq(Employee("Max", 22, 1, 100000.0), Employee("Adam", 33, 2, 93000.0), Employee("Eve", 35, 2, 89999.0), Employee("Muller", 39, 3, 120000.0))).toDS()
val employeeDataSet2 = sc.parallelize(Seq(Employee("John", 26, 1, 990000.0), Employee("Joe", 38, 3, 115000.0))).toDS()
val departmentDataSet = sc.parallelize(Seq(Department(1, "Engineering"), Department(2, "Marketing"), Department(3, "Sales"))).toDS()

defined class Employee
defined class Department
defined class Record
defined class ResultSet
spark2 = org.apache.spark.sql.SparkSession@36ca9a37
employeeDataSet1 = [name: string, age: int ... 2 more fields]
employeeDataSet2 = [name: string, age: int ... 2 more fields]
departmentDataSet = [id: int, name: string]


[id: int, name: string]

In [14]:
val employeeDataset = employeeDataSet1.union(employeeDataSet2)

employeeDataset = [name: string, age: int ... 2 more fields]


[name: string, age: int ... 2 more fields]

In [15]:
employeeDataset.show()

+------+---+------------+--------+
|  name|age|departmentId|  salary|
+------+---+------------+--------+
|   Max| 22|           1|100000.0|
|  Adam| 33|           2| 93000.0|
|   Eve| 35|           2| 89999.0|
|Muller| 39|           3|120000.0|
|  John| 26|           1|990000.0|
|   Joe| 38|           3|115000.0|
+------+---+------------+--------+



In [16]:
def averageSalary(key: (Int, String), iterator: Iterator[Record]): ResultSet = {
  val (total, count) = iterator.foldLeft(0.0, 0.0) {
      case ((total, count), x) => (total + x.salary, count + 1)
  }
  ResultSet(key._1, key._2, total/count)
}

val averageSalaryDataset = employeeDataset.joinWith(departmentDataSet, $"departmentId" === $"id", "inner")
                                          .map(record => Record(record._1.name, record._1.age, record._1.salary, record._1.departmentId, record._2.name))
                                          .filter(record => record.age > 25)
                                          .groupBy($"departmentId", $"departmentName")
                                          .avg()

averageSalaryDataset = [departmentId: int, departmentName: string ... 3 more fields]


averageSalary: (key: (Int, String), iterator: Iterator[Record])ResultSet


[departmentId: int, departmentName: string ... 3 more fields]

In [17]:
averageSalaryDataset.show()

+------------+--------------+--------+-----------+-----------------+
|departmentId|departmentName|avg(age)|avg(salary)|avg(departmentId)|
+------------+--------------+--------+-----------+-----------------+
|           2|     Marketing|    34.0|    91499.5|              2.0|
|           1|   Engineering|    26.0|   990000.0|              1.0|
|           3|         Sales|    38.5|   117500.0|              3.0|
+------------+--------------+--------+-----------+-----------------+



## Convert a Dataset to a DataFrame

In [18]:
import org.apache.spark.sql.functions._

val wordsDataset = sc.parallelize(Seq("Spark I am your father", "May the spark be with you", "Spark I am your father")).toDS()
val result = wordsDataset
              .flatMap(_.split(" "))               // Split on whitespace
              .filter(_ != "")                     // Filter empty words
              .map(_.toLowerCase())
              .toDF()                              // Convert to DataFrame to perform aggregation / sorting
              .groupBy($"value")                   // Count number of occurrences of each word
              .agg(count("*") as "numOccurances")
              .orderBy($"numOccurances" desc)      // Show most common words first


wordsDataset = [value: string]
result = [value: string, numOccurances: bigint]


[value: string, numOccurances: bigint]

In [19]:
result.show()

+------+-------------+
| value|numOccurances|
+------+-------------+
| spark|            3|
|father|            2|
|    am|            2|
|     i|            2|
|  your|            2|
|  with|            1|
|   you|            1|
|   may|            1|
|    be|            1|
|   the|            1|
+------+-------------+



# The END